# **An Intelligent AI Agent for Personalized Learning**

 Gen AI-powered learning assistant designed to help students study more effectively using their own materials.

 Built using LangChain v0.1+, Gemini, and modern GenAI capabilities, it enables personalized, grounded, and interactive academic support.

Summarize research papers into concise study notes
Answer personalized questions based on user-uploaded content using RAG (Retrieval-Augmented Generation)

Random fact = 1.76 trillion / 600,000 = 2,933,333.33.
1.76 trillion- Parameter trained
600000 - total words in oxford Dictionary

**We will implement :**



*   Summarize research papers into concise study notes
*   Answer personalized questions using RAG( **retrieval + reasoning**) over user content





I asked god why you made fear
sab rona chahte but we hate tears
I'm never sober but stay clear
samjh se bahar jaise shakespeare


# **High-level architecture (ScholarAI mental model**

User Manual / Abstracts

        ↓
Text Chunking

        ↓
Embeddings (SentenceTransformer)

        ↓
Vector Store (FAISS)

        ↓
Retriever

        ↓
LLM (Hugging Face)

        ↓
Grounded Answer / Summary


**Installing all the required libraries**

In [1]:
!pip install -U langchain-community langchain-core huggingface_hub faiss-cpu sentence-transformers streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.2/494.2 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.12
    Uninstalling langchain-core-1.2.12:
      Successfully uninstalled langchain-core-1.2.12
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformer

**Step-1 Load the Kaggle Dataset**

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("spsayakpaul/arxiv-paper-abstracts")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'arxiv-paper-abstracts' dataset.
Path to dataset files: /kaggle/input/arxiv-paper-abstracts


In [3]:
import os
os.listdir(path)

['arxiv_data_210930-054931.csv', 'arxiv_data.csv']

In [4]:
import pandas as pd

csv_path1=os.path.join(path,'arxiv_data.csv')
df1=pd.read_csv(csv_path1)
df1.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [5]:
df1['summaries'][0]

'Stereo matching is one of the widely used techniques for inferring depth from\nstereo images owing to its robustness and speed. It has become one of the major\ntopics of research since it finds its applications in autonomous driving,\nrobotic navigation, 3D reconstruction, and many other fields. Finding pixel\ncorrespondences in non-textured, occluded and reflective areas is the major\nchallenge in stereo matching. Recent developments have shown that semantic cues\nfrom image segmentation can be used to improve the results of stereo matching.\nMany deep neural network architectures have been proposed to leverage the\nadvantages of semantic segmentation in stereo matching. This paper aims to give\na comparison among the state of art networks both in terms of accuracy and in\nterms of speed which are of higher importance in real-time applications.'

In [6]:
csv_path2=os.path.join(path,"arxiv_data_210930-054931.csv")
df2=pd.read_csv(csv_path1)


df2.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [7]:
# We will rename to colummn "abstract" inorder to concat them
df2=df2.rename(columns={'abstracts':'summaries'})
df2.tail()

,titles,summaries,terms
51769,Hierarchically-coupled hidden Markov models fo...,We address the problem of analyzing sets of no...,"['stat.ML', 'physics.bio-ph', 'q-bio.QM']"
51770,Blinking Molecule Tracking,We discuss a method for tracking individual mo...,"['cs.CV', 'cs.DM']"
51771,Towards a Mathematical Foundation of Immunolog...,We attempt to set a mathematical foundation of...,"['stat.ML', 'cs.LG', 'q-bio.GN']"
51772,A Semi-Automatic Graph-Based Approach for Dete...,Diffusion Tensor Imaging (DTI) allows estimati...,['cs.CV']
51773,SparseCodePicking: feature extraction in mass ...,Mass spectrometry (MS) is an important techniq...,"['stat.ML', 'physics.med-ph', 'stat.AP', 'stat..."


In [8]:
# We will aslo conacat these 2 Dataframe into 1
df=pd.concat([df1,df2],ignore_index=True)
df.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [9]:
type(df['summaries'])

pandas.core.series.Series

In [10]:
df.shape

(103548, 3)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103548 entries, 0 to 103547
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   titles     103548 non-null  object
 1   summaries  103548 non-null  object
 2   terms      103548 non-null  object
dtypes: object(3)
memory usage: 2.4+ MB


**Step-2 Preparing Data for RAG**

In [12]:
import os
from dotenv import load_dotenv
#  .env file into Python's os.environ

load_dotenv()

os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')
os.environ['HUGGINGFACEHUB_API_TOKEN']=os.getenv('HF_Token')

In [13]:
# # sampling the dataset using only first 25 rows

# # docum="\n\n".join((df['titles'].astype(str) +"\n\n" +df['summaries'].astype(str))[:25].tolist())
# docum = "\n\n".join((df['titles'].astype(str) + "\n\n" + df['summaries'].astype(str)).iloc[:25].tolist())




# print(len(docum),type(docum))
# # len(docum)

In [14]:
df_small = df.sample(n=3000, random_state=42)

# docum= (
#     df_small["titles"].astype(str)
#     + "\n\n"
#     + df_small["summaries"].astype(str)
# ).tolist()

docum = (
    "Paper title: " + df_small["titles"].astype(str) +
    "\nMain contribution: " + df_small["summaries"].astype(str)
).tolist()


In [15]:
print(len(docum),type(docum))
docum[0]

3000 <class 'list'>


'Paper title: CAMul: Calibrated and Accurate Multi-view Time-Series Forecasting\nMain contribution: Probabilistic time-series forecasting enables reliable decision making across\nmany domains. Most forecasting problems have diverse sources of data containing\nmultiple modalities and structures. Leveraging information as well as\nuncertainty from these data sources for well-calibrated and accurate forecasts\nis an important challenging problem. Most previous work on multi-modal learning\nand forecasting simply aggregate intermediate representations from each data\nview by simple methods of summation or concatenation and do not explicitly\nmodel uncertainty for each data-view. We propose a general probabilistic\nmulti-view forecasting framework CAMul, that can learn representations and\nuncertainty from diverse data sources. It integrates the knowledge and\nuncertainty from each data view in a dynamic context-specific manner assigning\nmore importance to useful views to model a well-cali

In [16]:
# Adhura- Daku ,kya karein-Aditya Rikhari,Yadein - krazer,

**Step-3 Chunk the text**

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
    separators=["\n\n", "\n", ".", " "]
    )


docs = []
for text in docum:
    docs.extend(text_splitter.create_documents([text]))

len(docs)


6749

**Step-4 Creating embedding(HuggingFace)**

In [18]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings=HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/tmp/ipython-input-1717540386.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Step-5 Build Vectore Store(FAISS)**

In [19]:
# from langchain.vectorstores import FAISS
from langchain_community.vectorstores.faiss import FAISS

INDEX_PATH = "faiss_index"

if os.path.exists(INDEX_PATH):
    vectorstore = FAISS.load_local(INDEX_PATH, embeddings)
else:
    vectorstore = FAISS.from_documents(docs, embeddings)
    vectorstore.save_local(INDEX_PATH)

In [20]:
# from langchain_community.vectorstores.faiss import FAISS
# import os

# INDEX_PATH = "faiss_index"

# if os.path.exists(INDEX_PATH):
#     print("Loading existing vector store...")
#     vectorstore = FAISS.load_local(
#         INDEX_PATH,
#         embeddings,
#         allow_dangerous_deserialization=True
#     )
# else:
#     print("Building vector store (one-time)...")
#     vectorstore = FAISS.from_documents(docs, embeddings)
#     vectorstore.save_local(INDEX_PATH)


**Retriever**

In [21]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)


In [37]:
# #will query using our vectorestore database
query='''Machine learning solutions for pattern classification problems are nowadays
widely deployed in society and industry.'''
result=vectorstore.similarity_search_with_relevance_scores(query)
result[0]


#


(Document(id='6b289dc8-cfe4-4ef5-820b-3322ecca6c6f', metadata={}, page_content='semiconductor manufacturing data. The paper deals with two broad classes of\nproblems: engineering the features in a feature vector representation and\nengineering the definition of the target concept (the classes). Manufacturing\nprocess data present special problems for feature engineering, since the data\nhave multiple levels of granularity (detail, resolution). Engineering the\ntarget concept is important, due to our focus on understanding the past, as\nopposed to the more common focus in machine learning on predicting the future.'),
 np.float32(0.4054163))

**core is 0.405 → VALID CONTEXT**

**Step-6 Hugging Face LLM**

In [23]:
from transformers import pipeline

# Use a text-generation model
summarizer_llm = pipeline(
    "text-generation",
    model="google/flan-t5-base",
    max_new_tokens=200,
    do_sample=False
    )  # instruction-tuned model



# summarizer = pipeline(
#     "summarization",
#     model="facebook/bart-large-cnn",
#     device=0  # use GPU if available
# )


# llm=pipeline(
#     "text-generation",
#     model="mistralai/Mistral-7B-Instruct-v0.2",
#     max_new_tokens=400,
#     temperature=0.2,device_map="auto"
# )



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Passing `generation_config` together with generation-related arguments=({'max_new_tokens', 'do_sample'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM

**Step7 - Research Paper Summarization**

This produce study notes

In [24]:
def summarize_paper(text):
    prompt = (
        #"Summarize:\n"
        "summarize the following academic text in bullet points:\n\n"
        f"{text}"
    )

    output = summarizer_llm(prompt)
    return output[0]["generated_text"]













# def summarize_paper(text):
#     return summarizer_llm(
#         text,
#         max_length=180,
#         min_length=60,
#         do_sample=False
#     )[0]["summary_text"]


In [25]:
docum[0]

'Paper title: CAMul: Calibrated and Accurate Multi-view Time-Series Forecasting\nMain contribution: Probabilistic time-series forecasting enables reliable decision making across\nmany domains. Most forecasting problems have diverse sources of data containing\nmultiple modalities and structures. Leveraging information as well as\nuncertainty from these data sources for well-calibrated and accurate forecasts\nis an important challenging problem. Most previous work on multi-modal learning\nand forecasting simply aggregate intermediate representations from each data\nview by simple methods of summation or concatenation and do not explicitly\nmodel uncertainty for each data-view. We propose a general probabilistic\nmulti-view forecasting framework CAMul, that can learn representations and\nuncertainty from diverse data sources. It integrates the knowledge and\nuncertainty from each data view in a dynamic context-specific manner assigning\nmore importance to useful views to model a well-cali

In [26]:

print(summarize_paper(docum[0]))


Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


summarize the following academic text in bullet points:

Paper title: CAMul: Calibrated and Accurate Multi-view Time-Series Forecasting
Main contribution: Probabilistic time-series forecasting enables reliable decision making across
many domains. Most forecasting problems have diverse sources of data containing
multiple modalities and structures. Leveraging information as well as
uncertainty from these data sources for well-calibrated and accurate forecasts
is an important challenging problem. Most previous work on multi-modal learning
and forecasting simply aggregate intermediate representations from each data
view by simple methods of summation or concatenation and do not explicitly
model uncertainty for each data-view. We propose a general probabilistic
multi-view forecasting framework CAMul, that can learn representations and
uncertainty from diverse data sources. It integrates the knowledge and
uncertainty from each data view in a dynamic context-specific manner assigning
more imp

**RAG Question Answering model (retrieved context → answer)**

In [27]:
from transformers import pipeline

llm = pipeline(
    "text-generation",
    model="google/flan-t5-base",
    max_new_tokens=256
)


Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM

In [28]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)


In [58]:
def answer_question(question):
    docs = retriever.invoke(question)

    if not docs:
        return "Not found in the provided material."

    context = "\n\n".join(d.page_content for d in docs)

    prompt = f"""
    You are answering a question strictly from the given context.

    Context:
    {context}

    Question:
    {question}

    Rules:
    - If the answer is not in the context, say:
      "Not found in the provided material."
    - Be concise and factual.
    """

    result = llm(prompt)[0]["generated_text"]
    return result.strip()


In [59]:
print(answer_question("What is stereo matching used for?"))
# print(answer_question("What is the capital of France?"))


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


You are answering a question strictly from the given context.

    Context:
    window is proposed, and the disparity image is optimized by using left and
right consistency detection to further reduce the error matching rate. The
experimental results show that the proposed algorithm can effectively enhance
the stereo matching effect of the image providing significant improvement in
accuracy as compared with the classical census algorithm. The proposed model
code, dataset, and experimental results are available at
https://github.com/WangHewei16/Stereo-Matching.

window is proposed, and the disparity image is optimized by using left and
right consistency detection to further reduce the error matching rate. The
experimental results show that the proposed algorithm can effectively enhance
the stereo matching effect of the image providing significant improvement in
accuracy as compared with the classical census algorithm. The proposed model
code, dataset, and experimental results are availa

In [65]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.0 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.1
    Uninstalling cachetools-7.0.1:
      Successfully uninstalled cachetools-7.0.1


In [70]:
import sys
st.write("Python version:", sys.version)


2026-02-18 11:42:06.980 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-18 11:42:06.982 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-18 11:42:06.984 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-18 11:42:06.986 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-18 11:42:06.988 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-18 11:42:06.989 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [69]:
%%writefile app.py

import os
import streamlit as st

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import pipeline

# ----------------------------
# CONFIG
# ----------------------------
st.set_page_config(page_title="ScholarAI", layout="wide")

INDEX_PATH = "index"   # index.faiss + index.pkl must exist


# ----------------------------
# LOAD MODELS (ONCE)
# ----------------------------
@st.cache_resource
def load_models():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    llm = pipeline(
        "text-generation",
        model="google/flan-t5-base",
        max_new_tokens=256,
        do_sample=False
    )

    return embeddings, llm


embeddings, llm = load_models()


# ----------------------------
# LOAD VECTOR STORE (ONLY LOAD)
# ----------------------------
@st.cache_resource
def load_vectorstore():
    if not (
        os.path.exists("index.faiss")
        and os.path.exists("index.pkl")
    ):
        raise RuntimeError(
            "FAISS index not found. index.faiss and index.pkl must exist."
        )

    return FAISS.load_local(
        INDEX_PATH,
        embeddings,
        allow_dangerous_deserialization=True
    )


vectorstore = load_vectorstore()
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})


# ----------------------------
# CORE FUNCTIONS
# ----------------------------
def summarize_text(text):
    prompt = f"""
Summarize the following research content into concise study notes.

Rules:
- Bullet points
- Clear academic language
- No repetition
- No extra commentary

Text:
{text}
"""
    output = llm(prompt)[0]["generated_text"]
    return output.strip()


def answer_question(question):
    docs = retriever.invoke(question)

    if not docs:
        return "Not found in the provided material."

    context = "\n\n".join(d.page_content for d in docs)

    prompt = f"""
You are answering strictly from the given context.

Context:
{context}

Question:
{question}

Rules:
- If the answer is not in the context, say:
  "Not found in the provided material."
- Be concise and factual.
"""

    output = llm(prompt)[0]["generated_text"]
    return output.strip()


# ----------------------------
# STREAMLIT UI
# ----------------------------
st.title("📚 ScholarAI — Research Paper Assistant")

tab1, tab2 = st.tabs(["📝 Summarization", "❓ Q&A"])

with tab1:
    st.subheader("Summarize Research Content")

    input_text = st.text_area(
        "Paste paper content",
        height=250
    )

    if st.button("Summarize"):
        if not input_text.strip():
            st.warning("Please provide text.")
        else:
            with st.spinner("Summarizing..."):
                st.success(summarize_text(input_text))


with tab2:
    st.subheader("Ask Questions (RAG)")

    question = st.text_input("Enter your question")

    if st.button("Get Answer"):
        if not question.strip():
            st.warning("Please enter a question.")
        else:
            with st.spinner("Searching knowledge base..."):
                st.success(answer_question(question))


Overwriting app.py
